In [1]:
from Core import *
from configs import CONF

import geojson as gj
import pandas as pd

### Initial Filter

In [4]:
df = loadtab()
df = scale_filter(df)
df = season_filter(df)

#### Extract imIDs from GeoJSON

In [4]:
df = loadtab()
imIDs = extract_imIDs('season_filter', sv_txt = False)
df = df[df.PRODUCT_ID.isin(imIDs)]
df = cluster_centroids(df)

ctr1 = df[df.CLUSTER != -1].groupby(['CLUSTER'], as_index = False).agg({'CENTER': lambda x: get_centroid(x), 'PRODUCT_ID': lambda x: list(x)})
ctr2 = df[df.CLUSTER == -1][['CLUSTER', 'CENTER','PRODUCT_ID']]
ctr = pd.concat([ctr1, ctr2], axis=0).reset_index(drop=True)

# qgis_geojson_layer(df[df.CLUSTER != -1], 'img_stacks')
# qgis_geojson_layer(df[df.CLUSTER == -1], 'img_singls')
# qgis_geojson_layer(ctr, 'cluster_centroids', geom = 'point')

with open('clusters.txt', 'w') as f:
    for idx, val in ctr.iterrows():
        f.write(f"--- {{{idx}}} ---\n")
        if isinstance(val.PRODUCT_ID, list):
            for item in val.PRODUCT_ID:
                f.write("%s\n" % item)
        else:
            f.write("%s\n" % val.PRODUCT_ID)
        f.write("\n")